In [1]:
using BenchmarkTools

In [313]:
gensym(:scrabble)

Symbol("##scrabble#346")

In [359]:
module compile

macro feature(ex, description::String="")
    Expr(:call, :Feature, Expr(:quote, ex), Expr(:quote, Expr(:quote, description)))
end

macro feature(ex, description::Expr)
    Expr(:call, :Feature, Expr(:quote, ex), Expr(:quote, description))
end

function extract_cached_features!(setup::Vector{Expr}, rhs::Expr, input_var::Symbol)
    for (i, expr) in enumerate(rhs.args)
        if isa(expr, Expr)
            if expr.head == :call && length(expr.args) == 2 && expr.args[2] == input_var
                cached_varname = gensym(expr.args[1])
                push!(setup, :($cached_varname = $(copy(expr))))
                rhs.args[i] = cached_varname
            else
                extract_cached_features!(setup, expr, input_var)
            end
        end
    end
end

function extract_cached_features(expr::Expr, input_var::Symbol)
    setup = Expr[]
    rhs = copy(expr)
    extract_cached_features!(setup, rhs, input_var)
    Expr(:block, setup...), rhs
end

immutable Feature
    setup::Expr
    test::Expr
    args::Vector{Tuple{Symbol, Expr}}
    length::Int
    description::Expr
    function Feature(expr::Expr, description::Expr)
        rhs = expr.args[1]
        args = Tuple{Symbol, Expr}[]
        for ex in expr.args[2:end]
            @assert ex.head == :(=)
            push!(args, (ex.args[1], ex.args[2]))
        end
        len = prod(length(eval(ex[2])) for ex in args)
        setup, test = extract_cached_features(rhs, :word)
        new(setup, test, args, len, description)
    end
end
       
    immutable FeatureSet{F}
        features::Vector{Feature}
        evaluate::F
        descriptions::Vector{String}
    end
    
    function compile_test(features::Vector{Feature})
        num_features = sum([f.length for f in features])
        result = Expr(:block, [f.setup for f in features]...)
        push!(result.args, quote
            values = falses($(num_features))
            i = 1
        end)
        for (i, feat) in enumerate(features)
            inner = quote
                values[i] = $(feat.test)
                i += 1
            end
            for (var, expr) in reverse(feat.args)
              inner = quote
                    for $var in $expr
                        $inner
                    end
                end
            end
            push!(result.args, inner)
        end
        push!(result.args, :values)
        println(result)
        result
    end
    
    function compile_descriptions(features::Vector{Feature})
        num_features = sum([f.length for f in features])
        result = quote
            values = Vector{String}($(num_features))
            i = 1
        end
        for (i, feat) in enumerate(features)
            inner = quote
                values[i] = $(feat.description)
                i += 1
            end
            for (var, expr) in reverse(feat.args)
              inner = quote
                    for $var in $expr
                        $inner
                    end
                end
            end
            push!(result.args, inner)
        end
        push!(result.args, :values)
        result
    end

    function FeatureSet(features)
        test_expr = compile_test(features)
        description_expr = compile_descriptions(features)
        FeatureSet(features,
            eval(:((word) -> $(test_expr))),
            eval(:(() -> $(description_expr)))())
    end

    function scrabble_score(word::String)
        sum(scrabble_score(c) for c in word)
    end

    scrabble_score(char::Char) = SCRABBLE_SCORES[char]

    const SCRABBLE_SCORES = Dict{Char, Int}(
        'e' => 1,
        'a' => 1,
        'i' => 1,
        'o' => 1,
        'n' => 1,
        'r' => 1,
        't' => 1,
        'l' => 1,
        's' => 1,
        'u' => 1,
        'd' => 2,
        'g' => 2,
        'b' => 3,
        'c' => 3,
        'm' => 3,
        'p' => 3,
        'f' => 4,
        'h' => 4,
        'v' => 4,
        'w' => 4,
        'y' => 4,
        'k' => 5,
        'j' => 8,
        'x' => 8,
        'q' => 10,
        'z' => 10
    )

function allfeatures()
    features = Feature[]
    push!(features, @feature((scrabble_score(word) == j for j in 1:26), "scrabble score $j"))
    push!(features, @feature((c in word for c in 'a':'z'), "contains $c"))
    push!(features, @feature((length(word) >= j && word[j] == c for c in 'a':'z', j in 1:26), "contains $c at index $j"))
    features
end
    

end
    



compile

In [360]:
compile.allfeatures()

3-element Array{compile.Feature,1}:
 compile.Feature(quote 
    ##scrabble_score#360 = scrabble_score(word)
end,:(##scrabble_score#360 == j),Tuple{Symbol,Expr}[(:j,:(1:26))],26,:("scrabble score $(j)"))                 
 compile.Feature(quote 
end,:(c in word),Tuple{Symbol,Expr}[(:c,:('a':'z'))],26,:("contains $(c)"))                                                                                    
 compile.Feature(quote 
    ##length#361 = length(word)
end,:(##length#361 >= j && word[j] == c),Tuple{Symbol,Expr}[(:c,:('a':'z')),(:j,:(1:26))],676,:("contains $(c) at index $(j)"))

In [361]:
fset = compile.FeatureSet(compile.allfeatures())

begin 
    begin 
        ##scrabble_score#362 = scrabble_score(word)
    end
    begin 
    end
    begin 
        ##length#363 = length(word)
    end
    begin  # In[359], line 61:
        values = falses(728) # In[359], line 62:
        i = 1
    end
    begin  # In[359], line 71:
        for j = 1:26 # In[359], line 72:
            begin  # In[359], line 66:
                values[i] = ##scrabble_score#362 == j # In[359], line 67:
                i += 1
            end
        end
    end
    begin  # In[359], line 71:
        for c = 'a':'z' # In[359], line 72:
            begin  # In[359], line 66:
                values[i] = c in word # In[359], line 67:
                i += 1
            end
        end
    end
    begin  # In[359], line 71:
        for c = 'a':'z' # In[359], line 72:
            begin  # In[359], line 71:
                for j = 1:26 # In[359], line 72:
                    begin  # In[359], line 66:
                        values[i] = ##length#363 >= j && word

compile.FeatureSet{compile.##9#10}(compile.Feature[compile.Feature(quote 
    ##scrabble_score#362 = scrabble_score(word)
end,:(##scrabble_score#362 == j),Tuple{Symbol,Expr}[(:j,:(1:26))],26,:("scrabble score $(j)")),compile.Feature(quote 
end,:(c in word),Tuple{Symbol,Expr}[(:c,:('a':'z'))],26,:("contains $(c)")),compile.Feature(quote 
    ##length#363 = length(word)
end,:(##length#363 >= j && word[j] == c),Tuple{Symbol,Expr}[(:c,:('a':'z')),(:j,:(1:26))],676,:("contains $(c) at index $(j)"))],compile.#9,String["scrabble score 1","scrabble score 2","scrabble score 3","scrabble score 4","scrabble score 5","scrabble score 6","scrabble score 7","scrabble score 8","scrabble score 9","scrabble score 10"  …  "contains z at index 17","contains z at index 18","contains z at index 19","contains z at index 20","contains z at index 21","contains z at index 22","contains z at index 23","contains z at index 24","contains z at index 25","contains z at index 26"])

In [363]:
vals = fset.evaluate("hello")

728-element BitArray{1}:
 false
 false
 false
 false
 false
 false
 false
  true
 false
 false
 false
 false
 false
     ⋮
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false

In [366]:
[fset.descriptions[i] for i in 1:length(fset.descriptions) if vals[i]]

10-element Array{String,1}:
 "scrabble score 8"     
 "contains e"           
 "contains h"           
 "contains l"           
 "contains o"           
 "contains e at index 2"
 "contains h at index 1"
 "contains l at index 3"
 "contains l at index 4"
 "contains o at index 5"

In [367]:
word = "hello"
@benchmark $fset.evaluate($word)

BenchmarkTools.Trial: 
  memory estimate:  240.00 bytes
  allocs estimate:  4
  --------------
  minimum time:     8.050 μs (0.00% GC)
  median time:      8.264 μs (0.00% GC)
  mean time:        8.602 μs (0.00% GC)
  maximum time:     29.199 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     3
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [306]:
function manual(word)
    values = BitArray{1}()
    append!(values, BitArray{1}(c in word for c in 'a':'z'))
    append!(values, (length(word) >= j && word[j] == c for c in 'a':'z' for j in 1:26))
    values
#     i = 1
#     for v in (c in word for c in 'a':'z')
#         values[i] = v
#         i += 1
#     end
#     for v in (length(word) >= j && word[j] == c for c in 'a':'z' for j in 1:26)
#         values[i] = v
#         i += 1
#         end
#     values
end
    

manual (generic function with 1 method)

In [307]:
word = "hello"
@benchmark manual($word)

LoadError: LoadError: MethodError: Cannot `convert` an object of type Base.Generator{StepRange{Char,Int64},##52#55{String}} to an object of type BitArray{1}
This may have arisen from a call to the constructor BitArray{1}(...),
since type constructors fall back to convert methods.
while loading In[307], in expression starting on line 182